#### 짧은 영어 문장을 짧은 프랑스어 문장으로 번역하는 예제

##### 1. 라이브러리 로드 및 변수 초기화

In [1]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64  # 훈련을 위한 배치 크기
epochs = 100  # 훈련 할 에포크 수
latent_dim = 256  # 인코딩 공간의 잠재 된 차원 수
num_samples = 10000  # 훈련 할 샘플 수

data_path = 'fra-eng/fra.txt' # 데이터 경로

Using TensorFlow backend.


##### 2. 학습에 사용할 데이터 로드

In [2]:
# 데이터 백터화
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    # Tab을 시작 문자로 사용하고 \n 은 종료 문자
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

##### 3. 학습 대상 데이터 확인

In [3]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 92
Max sequence length for inputs: 16
Max sequence length for outputs: 59


##### 4. 번역에 사용될 문장들을 3차원 배열(encoder_input_data, decoder_input_data, decoder_target_data)로 초기화

In [4]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

# encoder_input_data는 (num_pairs, max_english_sentence_length, num_english_characters) 
# 형태의 3차원 배열로 영어 문장의 one-hot 형식 벡터 데이터로 초기화.
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
# decoder_input_data는 (num_pairs, max_french_sentence_length, num_french_characters)
# 형태의 3차원 배열로 불어 문장의 one-hot형식 벡터 데이터로 초기화.
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
# decoder_target_data는 decoder_input_data와 같지만
# 하나의 time step만큼 offset 됨. 
# decoder_target_data[:, t, :]는 decoder_input_data[:, t + 1, :]와 동일.
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

##### 5. 문장들을 3차원 배열(encoder_input_data, decoder_input_data, decoder_target_data)로 변환.

In [5]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data는 한 번의 단계로 decoder_input_data보다 앞서 있음.
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

##### 6. 기본 LSTM 기반의 encoder 생성

In [6]:
# 입력 시퀀스의 정의와 처리
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# `encoder_outputs`는 버리고 상태(`state_h, state_c`)는 유지
encoder_states = [state_h, state_c]

Instructions for updating:
Colocations handled automatically by placer.


##### 7. 기본 LSTM 기반의 decoder 생성

In [7]:
# `encoder_states`를 초기 상태로 사용해 decoder를 설정
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# 전체 출력 시퀀스를 반환하고 내부 상태도 반환하도록 decoder를 설정. 
# 학습 모델에서 상태를 반환하도록 하진 않지만, inference에서 사용할 예정.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

##### 8. 기본 LSTM 기반의 Seq2Seq model을 주어진 encoder_input_data와 decoder_input_data로 decoder_target_data를 예측. (해당 model은 teacher forcing을 사용)

In [8]:
# `encoder_input_data`와 `decoder_input_data`를 `decoder_target_data`로 반환하도록 모델을 정의
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# 학습 실행
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# 모델 저장
model.save('s2s.h5')

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 29s 4ms/step - loss: 1.2153 - acc: 0.7228 - val_loss: 1.0825 - val_acc: 0.6984
Epoch 2/100
8000/8000 [==============================] - 29s 4ms/step - loss: 0.8680 - acc: 0.7634 - val_loss: 0.8496 - val_acc: 0.7676
Epoch 3/100
8000/8000 [==============================] - 29s 4ms/step - loss: 0.6902 - acc: 0.8058 - val_loss: 0.7166 - val_acc: 0.7887
Epoch 4/100
8000/8000 [==============================] - 29s 4ms/step - loss: 0.5928 - acc: 0.8276 - val_loss: 0.6503 - val_acc: 0.8075
Epoch 5/100
8000/8000 [==============================] - 29s 4ms/step - loss: 0.5434 - acc: 0.8406 - val_loss: 0.6119 - val_acc: 0.8190
Epoch 6/100
8000/8000 [==============================] - 29s 4ms/step - loss: 0.5049 - acc: 0.8519 - val_loss: 0.5735 - val_acc: 0.8290
Epoch 7/10

8000/8000 [==============================] - 29s 4ms/step - loss: 0.0982 - acc: 0.9693 - val_loss: 0.5574 - val_acc: 0.8753
Epoch 57/100
8000/8000 [==============================] - 29s 4ms/step - loss: 0.0958 - acc: 0.9701 - val_loss: 0.5598 - val_acc: 0.8759
Epoch 58/100
8000/8000 [==============================] - 29s 4ms/step - loss: 0.0938 - acc: 0.9704 - val_loss: 0.5646 - val_acc: 0.8748
Epoch 59/100
8000/8000 [==============================] - 29s 4ms/step - loss: 0.0912 - acc: 0.9712 - val_loss: 0.5680 - val_acc: 0.8759
Epoch 60/100
8000/8000 [==============================] - 29s 4ms/step - loss: 0.0890 - acc: 0.9719 - val_loss: 0.5725 - val_acc: 0.8746
Epoch 61/100
8000/8000 [==============================] - 30s 4ms/step - loss: 0.0871 - acc: 0.9725 - val_loss: 0.5804 - val_acc: 0.8750
Epoch 62/100
8000/8000 [==============================] - 29s 4ms/step - loss: 0.0848 - acc: 0.9732 - val_loss: 0.5867 - val_acc: 0.8747
Epoch 63/100
8000/8000 [==============================

C:\Users\user\Anaconda3\lib\site-packages\keras\engine\network.py:877: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


##### 9. 샘플링 할 encoder, decoder 모델 정의

In [9]:
# 샘플링 모델 정의
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state = decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

##### 10. model이 정상적으로 작동하는지 확인하기 위한 색인 생성

In [10]:
# 시퀀스를 다시 해독하기 위한 역방향 조회 토큰 색인
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

##### 11. 문장 Decoding을 위한 함수 (model이 정상적으로 작동하는지 확인을 위한 함수)

In [11]:
def decode_sequence(input_seq):
    # 상태 벡터로서 입력값을 encode
    states_value = encoder_model.predict(input_seq)

    # 길이가 1인 빈 목표 시퀀스를 생성
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # 대상 시퀀스 첫 번째 문자를 시작 문자로 기재.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # 시퀀스들의 batch에 대한 샘플링 반복(간소화를 위해, 배치 크기는 1로 상정)
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # 토큰으로 샘플링
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # 탈출 조건 : 최대 길이에 도달하거나
        # 종료 문자를 찾을 경우
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # (길이 1인) 목표 시퀀스 업데이트
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # 상태 업데이트
        states_value = [h, c]

    return decoded_sentence

##### 12. encoder_input_data의 샘플을 decoder_target_data의 표본으로 변환 (model이 정상적으로 작동하는지 확인)

In [12]:
for seq_index in range(100):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Go.
Decoded sentence: Va !

-
Input sentence: Hi.
Decoded sentence: Salut.

-
Input sentence: Hi.
Decoded sentence: Salut.

-
Input sentence: Run!
Decoded sentence: Courez !

-
Input sentence: Run!
Decoded sentence: Courez !

-
Input sentence: Who?
Decoded sentence: Qui ?

-
Input sentence: Wow!
Decoded sentence: Ça alors !

-
Input sentence: Fire!
Decoded sentence: Au feu !

-
Input sentence: Help!
Decoded sentence: À l'aide !

-
Input sentence: Jump.
Decoded sentence: Saute.

-
Input sentence: Stop!
Decoded sentence: Stop !

-
Input sentence: Stop!
Decoded sentence: Stop !

-
Input sentence: Stop!
Decoded sentence: Stop !

-
Input sentence: Wait!
Decoded sentence: Attends !

-
Input sentence: Wait!
Decoded sentence: Attends !

-
Input sentence: Go on.
Decoded sentence: Poursuivez.

-
Input sentence: Go on.
Decoded sentence: Poursuivez.

-
Input sentence: Go on.
Decoded sentence: Poursuivez.

-
Input sentence: Hello!
Decoded sentence: Salut !

-
Input sentence: Hello